In [7]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import os
import json
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score

In [8]:
amzn_2grams = []
amzn_3grams = []
amzn_ngrams = []
aapl_2grams = []
aapl_3grams = []
aapl_ngrams = []
config = json.load(open("../../configure.json"))
path = config["project_dir"] + config["feature_pkls"]
pkls = os.listdir(path)
results = []
for pkl in tqdm(pkls):
    data = pd.read_pickle(path + pkl, compression='infer')
    train_x = np.stack(data['features'])
    train_y = np.stack(data['trend'].astype('int'))

    pca = PCA(n_components=200)
    pComponents = pca.fit_transform(train_x)
    x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)
    xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
    xgb_model.fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    names = pkl.split("_", )
    pkl_save_path = config['project_dir'] + "models/xgboost/pkls/ngram/"
    model_name = "xg_" + pkl
    with open(pkl_save_path + model_name, 'wb') as f:
        pickle.dump(xgb_model, f)
    result = [names[1], round(acc * 100, 2), round(f1 * 100, 2)]
    if "amzn" == names[0]:
        if "2" in names[2]:
            amzn_2grams.append(result)
        if "3" in names[2]:
            amzn_3grams.append(result)
        if "n" in names[2]:
            amzn_ngrams.append(result)
    if "aapl" == names[0]:
        if "2" in names[2]:
            aapl_2grams.append(result)
        if "3" in names[2]:
            aapl_3grams.append(result)
        if "n" in names[2]:
            aapl_ngrams.append(result)

  0%|          | 0/36 [00:02<?, ?it/s]


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [-1  1]

In [ ]:
print("amazon 2grams results\n", amzn_2grams)
print("amazon 3grams results\n", amzn_3grams)
print("apple 2grams results\n", aapl_2grams)
print("apple 3grams results\n", aapl_3grams)

amazon 2grams results
 [['30', '0.698063'], ['60', '0.713028'], ['240', '0.675176'], ['5', '0.700704'], ['1440', '0.699824'], ['15', '0.691021']]
amazon 3grams results
 [['60', '0.637324'], ['30', '0.620599'], ['240', '0.638204'], ['15', '0.635563'], ['1440', '0.610915'], ['5', '0.630282']]
apple 2grams results
 [['15', '0.635578'], ['30', '0.653393'], ['60', '0.661514'], ['240', '0.649201'], ['1440', '0.667540'], ['5', '0.663348']]
apple 3grams results
 [['15', '0.592350'], ['1440', '0.598900'], ['240', '0.594184'], ['60', '0.634530'], ['30', '0.606497'], ['5', '0.623788']]
